In [10]:
!pip install pymc arviz

In [3]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import pymc as pm
import arviz as az

# Load data (update path as needed)
try:
    data = pd.read_csv('../Classified Data.csv')  # Update this path if needed
except FileNotFoundError:
    print("Data file not found. Please check the path and filename.")
    data = None

if data is not None:
    # Example: create dummy columns if not present (for demonstration)
    if 'county' not in data.columns:
        data['county'] = 0
    if 'county_code' not in data.columns:
        data['county_code'] = 0
    if 'floor' not in data.columns:
        data['floor'] = 0
    if 'log_radon' not in data.columns:
        data['log_radon'] = 0

    county_names = data.county.unique()
    county_idx = data['county_code'].values

    with pm.Model() as hierarchical_model:
        # Hyperpriors
        mu_a = pm.Normal('mu_alpha', mu=0., sigma=100**2)
        sigma_a = pm.Uniform('sigma_alpha', lower=0, upper=100)
        mu_b = pm.Normal('mu_beta', mu=0., sigma=100**2)
        sigma_b = pm.Uniform('sigma_beta', lower=0, upper=100)

        # Intercept for each county, distributed around group mean mu_a
        a = pm.Normal('alpha', mu=mu_a, sigma=sigma_a, shape=len(county_names))
        # Slope for each county, distributed around group mean mu_b
        b = pm.Normal('beta', mu=mu_b, sigma=sigma_b, shape=len(county_names))

        # Model error
        eps = pm.Uniform('eps', lower=0, upper=100)

        # Expected value
        radon_est = a[county_idx] + b[county_idx] * data.floor.values

        # Data likelihood
        y_like = pm.Normal('y_like', mu=radon_est, sigma=eps, observed=data.log_radon)

        # Sample from the posterior
        hierarchical_trace = pm.sample(2000, tune=1000, target_accept=0.9, return_inferencedata=True)

    az.plot_trace(hierarchical_trace)
    plt.show()

Data file not found. Please check the path and filename.
